In [ ]:
var assetList = ee.data.listAssets("projects/ee-mvgonzalez/assets/NTL")['assets']
                    .map(function(d) { return d.name })
var ntl = ee.ImageCollection(assetList)
///print(ntl)
                 
///Map.addLayer(region)
var ciu_ntl = ntl.filterBounds(region)

function clp(img) {
  return img.clip(region)
}

var urban = ciu_ntl.map(clp)

var urb = urban.select('b1')


// Uniquely label the hotspot image objects.

var urb = urb.limit(1, 'system:time_start', false).first()
var rad = urb.select('b1')
var urb = rad.updateMask(rad.gt(20))

var palettes = require('users/gena/packages:palettes');
var palette = palettes.colorbrewer.RdYlGn[9]; 

// applies thresholds and computes new property to be used for filtering
///agrego radiance como float a urban de nuevo 

var addVariables = function(image) {
  // Compute time in fractional years since the epoch.
  var rad = image.select('b1')
  var rad = rad.updateMask(rad.gt(20))
  // Return the image with the added bands.
  return image
    // Add a time band.
    .addBands(ee.Image(rad).rename('radiance'))
    ;
};

var med = urban.map(addVariables)
var median = med.select('radiance')

// Define time range, agrego variable años
// FULL TS
var add_bands = function(image) {
  image = ee.Image(image);
  return image.addBands(ee.Image.constant(ee.Number.parse(image.date().format("Y"))).rename('year').float());
};

// Reduce to vectors
var collection = median.map(add_bands)


var start_t =  collection.map(
  function(img) {
  var rad = img.select('radiance')
  return img.updateMask(rad.gt(0))
}).select('year').reduce(ee.Reducer.min()).rename('inicial')

var start_t = start_t.select(start_t.bandNames())

//print(start_t.getInfo())


//Map.addLayer(start_t.select('inicial'), {min: 2000, max: 2020, palette: ['00FFFF', '0000FF']})


var fin =  collection.map(
  function(img) {
  var rad = img.select('radiance')
  return img.updateMask(rad.lt(64))
}).select('year').reduce(ee.Reducer.max()).rename('final')

var fin = fin.select(fin.bandNames())



//Map.addLayer(fin.select('final'), {min: 2000, max: 2021, palette: ['00FFFF', '0000FF']})

//tasa de cambio
var time = fin.select('final').subtract(start_t.select('inicial')).rename('tiempo')
var time = time.updateMask(time.select('tiempo').gt(0))
//Map.addLayer(time, {min: 1, max: 21, palette: ['00FFFF', '0000FF'],})

var first_img = median.map(
  function(img) {
  var rad = img.select('radiance')
  return img.updateMask(rad.gt(0))
}).select('radiance').reduce(ee.Reducer.firstNonNull()).rename('radiance')

//Map.addLayer(first_img)

var last_img = median.sort('system:time_start', false).map(
  function(img) {
  var rad = img.select('radiance')
  return img.updateMask(rad.gt(0))
}).select('radiance').reduce(ee.Reducer.firstNonNull()).rename('radiance')

var palettes = require('users/gena/packages:palettes');
var palette = palettes.colorbrewer.RdYlGn[9];
var dmsp_slope = (last_img.select('radiance').subtract(first_img.select('radiance'))).divide(time).rename('tasa')

Map.addLayer(dmsp_slope , {palette: palette,})

///clasifico a nivel de pixel

var ima_last = median.select('radiance').sum();

var ima_stack = ima_last.addBands(start_t)
var ima_stack = ima_stack.addBands(fin)
var ima_stack = ima_stack.addBands(time)
var ima_stack = ima_stack.addBands(dmsp_slope)
///print(ima_stack)



//EXPORTO EL STACK
Export.image.toDrive({
  image: ima_stack,
  region: region,
  description: 'stack_III',
  scale:1000,
  fileFormat: 'GeoTIFF',
  skipEmptyTiles: true,
  maxPixels: 1e13
});